In [28]:
## this version is for the format: e.g. 1P22, 1endp22

import numpy as np
from math import floor
import music21
import re
import os

velocity = [0, 35, 45, 60, 80, 95, 105]
n = len(velocity)
vel_dic = dict(zip(list(range(n)),velocity))

def get_file(txt):
    with open(txt, 'r') as file:
        string = file.read()
        return string


def arrToStreamNotewise(score, sample_freq, note_offset, tempo):    
    # define variables and lists needed for chord decoding
    speed = 1./sample_freq
    piano_notes = []
    violin_notes = []
    time_offset = 0
    
    i = 0
    while i < len(score):
        if score[i][1:10] == "p_octave_":
            add_wait=""
            if score[i][-3:] == "eoc":
                add_wait = "eoc"
                score[i] = score[i][:-3]
            this_note = score[i][10:]
            score[i] = "p"+this_note
            score.insert(i+1, "p" + str(int(this_note) + 12) + add_wait)
            i += 1
        i += 1

    for i in range(len(score)):
        if score[i] in ["", " ", "<eos>", "<unk>"]:
            continue
        elif score[i][1:4] == "end":
            if score[i][-3:] == "eoc":
                time_offset += 1
            continue
        elif score[i][:4] == "wait":
            time_offset += int(score[i][4:])
            continue
        else:
            # Look ahead to see if an end<noteid> was generated
            # soon after.  
            duration = 1
            has_end = False
            note_string_len = len(score[i])
            for j in range(1, 200):
                if i+j == len(score):
                    break
                if score[i+j][:4] == "wait":
                    duration += int(score[i+j][4:])
                if score[i+j][1:3+note_string_len] == "end" + score[i][1:] or score[i+j][1:note_string_len]==score[i][1:]:
                    has_end = True
                    break
                if score[i+j][-3:] == "eoc":
                    duration += 1

            if not has_end:
                duration = 12

            add_wait = 0
            if score[i][-3:] == "eoc":
                score[i] = score[i][:-3]
                add_wait = 1

            try: 
                new_note = music21.note.Note(int(score[i][2:]) + note_offset)    
                new_note.duration = music21.duration.Duration(duration * speed)
                new_note.offset = time_offset*speed
                new_note.volume.velocity = vel_dic[int(score[i][0])]
                if score[i][1] == "v":
                    violin_notes.append(new_note)
                else:
                    piano_notes.append(new_note)                
            except:
                print("Unknown note: " + score[i])
                
            time_offset += add_wait


    # list of all notes for each instrument should be ready at this stage

    # creating music21 instrument objects      
    violin = music21.instrument.fromString("Violin")
    piano = music21.instrument.fromString("Piano")

    # insert instrument object to start (0 index) of notes list
    violin_notes.insert(0, violin)
    piano_notes.insert(0, piano)

    # create music21 stream object for individual instruments
    violin_stream = music21.stream.Stream(violin_notes)
    piano_stream = music21.stream.Stream(piano_notes)

    # merge both stream objects into a single stream of 2 instruments
    note_stream = music21.stream.Stream([violin_stream, piano_stream])

    # add tempo
    note_stream.insert(0.0,music21.tempo.MetronomeMark('maestoso', tempo))

    return note_stream

def write_midi(s, filename, output_folder):
    fp = s.write('midi', fp=output_folder+filename+".mid")
    
def decoding_music(notetxt,notename,sample_freq,note_offset,tempo):
    
    notestring = get_file(notetxt)

    score_note = notestring.split(" ")
    
    note_stream = arrToStreamNotewise(score_note, sample_freq, note_offset, tempo)

    pattern = re.compile('.*?/(.*?).txt')
    notename = re.findall(pattern, notetxt)[0]
    #notename = notetxt[23:-4]
    
    write_midi(note_stream, notename, note_output)
    
note_output = "output_midi/Requiem/"
sample_freq = 4
note_offset = 33
tempo = 160
notetxt = 'Requiem/lacrimosa_finished_volume_20epoch.txt'
decoding_music(notetxt, note_output, sample_freq, note_offset, tempo)


In [8]:
import os
txtfiles = os.listdir('training_output/samples')
notetxt = 'training_output/samples'+ txtfiles[0]


In [6]:
txtfiles

['result_epoch30_batch20.txt',
 'result_epoch29_batch120.txt',
 'result_epoch22_batch120.txt',
 'result_epoch13_batch140.txt',
 'result_epoch24_batch80.txt',
 'result_epoch6_batch40.txt',
 'result_epoch23_batch160.txt',
 'result_epoch33_batch40.txt',
 'result_epoch31_batch80.txt',
 'result_epoch1_batch80.txt',
 'result_epoch35_batch0.txt',
 'result_epoch33_batch60.txt',
 'result_epoch39_batch0.txt',
 'result_epoch14_batch80.txt',
 'result_epoch0_batch60.txt',
 'result_epoch0_batch40.txt',
 'result_epoch9_batch20.txt',
 'result_epoch25_batch40.txt',
 'result_epoch8_batch80.txt',
 'result_epoch45_batch0.txt',
 'result_epoch32_batch40.txt',
 'result_epoch4_batch140.txt',
 'result_epoch35_batch160.txt',
 'result_epoch24_batch140.txt',
 'result_epoch19_batch60.txt',
 'result_epoch13_batch80.txt',
 'result_epoch41_batch20.txt',
 'result_epoch43_batch60.txt',
 'result_epoch42_batch40.txt',
 'result_epoch43_batch120.txt',
 'result_epoch3_batch60.txt',
 'result_epoch33_batch20.txt',
 'result_ep

In [11]:
sample_freq = 12
note_offset = 33
tempo = 80
txtfiles = os.listdir('training_output/samples')
note_output = "output_midi/"
for i in range(len(txtfiles)):
    if txtfiles[i] != '.ipynb_checkpoints':
        notetxt = 'training_output/samples/'+ txtfiles[i]
        decoding_music(notetxt, note_output, sample_freq, note_offset, tempo)
